In [56]:
import numpy as np

#Self Attention In Depth 

In [57]:
# In attention block we have three key factors.
# query, key and value 
# to know what does query, key and value pls check Andrew Ng's transformer video
# Now lets go through all the above in more depth
# Example sentence 'what is self attention' this is having 4 words
# lets take a particular word say "attention"
# now "attention" can be represented as embedding vector  of any dimension, here we
# are taking it as 20

x3 = np.random.randn(1, 20)

In [58]:
# attention calculation for x3

# We will only focus on x3
# for x3 we need the query vector x3 
wquery_3 = np.random.randn(20, 20)
key = np.random.randn(4, 20) # 4 is the vocab size here i.e. len(['what', 'is', 'self', 'attention'])
value = np.random.randn(4, 20)
query_3 = np.dot(x3, wquery_3)  # query vector derived from x3 (1, 20)


# In order to get the raw attention vale for x3 we need to 
# dot product of query3 with all the keys from other words

score = []

# we will collect the keys from each word to know the information it has and multiply it
# with question we have 
# key roughly speaking is like answer so dot product of (question and answer) give you 
# amount information the previous or next word contains
# values can be treated as reward of Reinfocement Learning

# just to make it more intutive we name query_3 as question
question = query_3
for idx in range(4):
    answer = key[idx, :]
    inforamtion_contains = np.dot(question, answer.T)
    score.append(inforamtion_contains)

score = np.array(score)
print(score.shape)


(4, 1)


In [59]:
# score is not scaled 
print(score)

[[ 12.24002923]
 [-23.12175369]
 [-22.70992041]
 [-16.78562033]]


In [60]:
# In order to scale the score we will use softmax function
prob = np.exp(score) / np.sum(np.exp(score), axis=1, keepdims=True)

In [61]:
print(prob)

[[4.82107426e+02]
 [7.90399223e-22]
 [2.64721706e-22]
 [2.36825812e-13]]


In [62]:
# prob[1] represents how much word3 should give attention to word2
print(prob[1].item())

7.903992231186865e-22


In [65]:
# final step
# here we will aggregate the probs with reward
A3 = np.zeros((1, 20))
for idx in range(4):
    reward = value[idx, :]
    A3 += prob[idx] * reward

print(A3.shape)
print(A3)

# This is A3 is the single head attention value for the word "X3"

(1, 20)
[[  107.24998971  -735.93959195  -571.47619008    38.5024471
    262.14588853  -175.60632075  -527.2970708   -329.55485186
    108.84143762   -53.4805704    379.85877869   113.70570607
   -139.24381018   410.97271662  -517.41667149   207.87093945
   -105.60227675   362.96363259   200.23125176 -1498.35993269]]


In [68]:
# Now the above is only done for x3 which is attention 
# lets do it for all others

x = np.random.randn(4, 20) # as 4 is the vocab size and 20 is embedding size
w_query = np.random.rand(20, 20)
w_key = np.random.rand(20, 20)
w_value = np.random.randn(20, 20)

In [82]:
# first we will create an inefficient implementation of self attention

A = [None for _ in range(4)] # a list which will contain all attentions


key = np.dot(x, w_key) # 4 is the vocab size here i.e. len(['what', 'is', 'self', 'attention'])
value = np.dot(x, w_value)

vocab_size = 4 
for idx in range(vocab_size):
    score = []
    for j in range(vocab_size):
        query = np.dot(x[idx, :].reshape(1, -1), w_query)
        answer = key[idx, :]
        inforamtion_contains = np.dot(query, answer)
        score.append(inforamtion_contains)
    score = np.array(score)
    prob = np.exp(score) / np.sum(np.exp(score), axis=1, keepdims=True)


    # final step
    # here we will aggregate the probs with reward
    attention = np.zeros((1, 20))
    for k in range(vocab_size):
        reward = value[k, :]
        attention += prob[k] * reward
    A[idx] = attention

A = np.array(A).reshape(vocab_size, -1)
print(A.shape)

# Here A is containing all attention representaions of all the words in vocab
    

(4, 20)


In [83]:
# Now lets do all the above in a vectorized implementation
x = np.random.rand(4, 20)

w_query = np.random.rand(20, 20)
w_key = np.random.rand(20, 20)
w_value = np.random.randn(20, 20)


query = np.dot(x, w_query)
key = np.dot(x, w_key)
value = np.dot(x, w_value)

# Now instead of for loop, we will use np.dot for every next operations
score = np.dot(query, key.T) 
probs = np.exp(score) / np.sum(np.exp(score), axis=1, keepdims=True)

A = np.dot(probs, value)
print(A.shape)

(4, 20)
